# Build G matrix for the Greenland traction inversion: 
## <font color=blue>"build_Gmatrix.ipynb"</font>
#### Jul 18, 2022  <font color=red>(v. testing)</font>
##### Jeonghyeop Kim (jeonghyeop.kim@gmail.com)


1. A G-matrix will be built using basis functions. 
2. And then the G-matrix will be save in a file.  

In [26]:
########Import modules
import numpy as np
import pandas as pd
import geopandas as gpd

import warnings
warnings.filterwarnings('ignore')

In [27]:
######## How Many Basis Functions 
HowManyBasisFunctions=np.loadtxt("geometry_info.txt", skiprows=1)
HowManyCell=int(HowManyBasisFunctions[1])
HowManyCell = 172
# Output files
outputFILE_G_matrix="G_matrix.out" 

# `STEP 1:` **BUILD G-Matrix, $\bar{\bar{G}}$**

In [29]:
input_sample = "./FILES_basis_functions/vel_horizontal_basis_functions_1_1.gmt"
sample = np.loadtxt(input_sample)
data_length=len(sample)*2

In [30]:
input_sample_strain = "./FILES_basis_functions/average_strain_RECTANGULAR_1_1.out"
# sample = np.loadtxt(input_sample_strain)
df_strain = pd.read_table(input_sample_strain, sep="\s+", header=None)

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf = gpd.GeoDataFrame(df_strain, geometry=gpd.points_from_xy(df_strain.iloc[:, [2]], df_strain.iloc[:, [1]]))
gdf.crs = {'init' :'epsg:4326'}
result = gpd.sjoin(gdf, world, how='left')

df1 = pd.DataFrame(result)
rslt_df = df1[df1["iso_a3"]=="GRL"]
midpoints_on_greenland = rslt_df.iloc[:, 0:9]
midpoints_on_greenland = rslt_df.iloc[:, 0:9]
print(midpoints_on_greenland)
# df_subset = rslt_df[['LONG','LAT', 'EAST', 'NORTH', 'SE', 'SN', 'CORRE']]
# df_subset

       0     1     2         3         4         5    6    7    8
421  422  61.0 -47.0 -0.001589  0.000400 -0.001146  0.0  0.0  0.0
422  423  61.0 -45.0 -0.001613  0.000454 -0.001000  0.0  0.0  0.0
423  424  61.0 -43.0 -0.001620  0.000488 -0.000865  0.0  0.0  0.0
460  461  63.0 -49.0 -0.001231  0.000066 -0.001365  0.0  0.0  0.0
461  462  63.0 -47.0 -0.001318  0.000174 -0.001239  0.0  0.0  0.0
..   ...   ...   ...       ...       ...       ...  ...  ...  ...
868  869  83.0 -33.0 -0.000212 -0.000334 -0.000305  0.0  0.0  0.0
869  870  83.0 -31.0 -0.000241 -0.000307 -0.000320  0.0  0.0  0.0
870  871  83.0 -29.0 -0.000270 -0.000279 -0.000332  0.0  0.0  0.0
871  872  83.0 -27.0 -0.000300 -0.000250 -0.000342  0.0  0.0  0.0
872  873  83.0 -25.0 -0.000330 -0.000221 -0.000350  0.0  0.0  0.0

[172 rows x 9 columns]


In [31]:
# midpoints_on_greenland.to_csv("midpoints_on_greenland.csv")

In [32]:
df_G = pd.DataFrame(index = range(data_length)) 
names = ['lon','lat','ve','vn','se','sn','corr']
# Make a blank G matrix part related to Boundary Condition on data points

for i in midpoints_on_greenland.iloc[:, 0]:
        inputfile_exx = "./FILES_basis_functions/vel_horizontal_basis_functions_"+str(i)+"_1"+".gmt" 
        #exx horizontal
        inputfile_eyy = "./FILES_basis_functions/vel_horizontal_basis_functions_"+str(i)+"_2"+".gmt" 
        #eyy horizontal
        inputfile_exy = "./FILES_basis_functions/vel_horizontal_basis_functions_"+str(i)+"_3"+".gmt" 
        #exy horizontal 

        df_exx=pd.read_csv(inputfile_exx, header=None, sep=r'(?:,|\s+)', 
                            comment='#', engine='python')
        df_eyy=pd.read_csv(inputfile_eyy, header=None, sep=r'(?:,|\s+)', 
                            comment='#', engine='python')
        df_exy=pd.read_csv(inputfile_exy, header=None, sep=r'(?:,|\s+)', 
                            comment='#', engine='python')   

        # CHANGE the column names 
        df_exx.columns = names
        df_eyy.columns = names
        df_exy.columns = names

        #BUILD a column vector Gexx (i)

        df_exx_x = df_exx.iloc[:,[0,1,2]]  # saved vx basis function on the data points
        df_exx_y = df_exx.iloc[:,[0,1,3]]  # saved vn basis function on the data points

        df_exx_x=df_exx_x.rename(columns ={'ve': 'velo'}) #column name change
        df_exx_y=df_exx_y.rename(columns ={'vn': 'velo'}) #column name change
        
        # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
        df_exx_x=df_exx_x.sort_values(['lat', 'lon'], ascending=[True, True])
        df_exx_y=df_exx_y.sort_values(['lat', 'lon'], ascending=[True, True])
        
        # MERGE two columns (n*1) into a new column (2n*1)
        # > ignore_index = True : 
        # >   have one continuous index numbers,
        # >     ignorning each of the two dfs original indices   
        frames_Gexx=[df_exx_x,df_exx_y]
        df_Gexx=pd.concat(frames_Gexx,ignore_index=True) # merge the two dataFrames into one

        # BUILD a column vector Geyy (i)

        df_eyy_x = df_eyy.iloc[:,[0,1,2]]  # saved vx basis function on the data points
        df_eyy_y = df_eyy.iloc[:,[0,1,3]]  # saved vn basis function on the data points

        df_eyy_x=df_eyy_x.rename(columns ={'ve': 'velo'}) #column name change
        df_eyy_y=df_eyy_y.rename(columns ={'vn': 'velo'}) #column name change

        # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
        df_eyy_x=df_eyy_x.sort_values(['lat', 'lon'], ascending=[True, True])
        df_eyy_y=df_eyy_y.sort_values(['lat', 'lon'], ascending=[True, True])
        
        # MERGE two columns (n*1) into a new column (2n*1)
        # > ignore_index = True : 
        # >   have one continuous index numbers,
        # >     ignorning each of the two dfs original indices
        frames_Geyy=[df_eyy_x,df_eyy_y]
        df_Geyy=pd.concat(frames_Geyy,ignore_index=True) # merge the two dataFrames into one
        

        # BUILD a column vector Gexy (i)

        df_exy_x = df_exy.iloc[:,[0,1,2]]  # saved vx basis function on the data points
        df_exy_y = df_exy.iloc[:,[0,1,3]]  # saved vn basis function on the data points

        df_exy_x=df_exy_x.rename(columns ={'ve': 'velo'}) #column name change
        df_exy_y=df_exy_y.rename(columns ={'vn': 'velo'}) #column name change

    
        # !! SORT VALUES !! # lat (ascending) first, and then lon (ascending).
        df_exy_x=df_exy_x.sort_values(['lat', 'lon'], ascending=[True, True])
        df_exy_y=df_exy_y.sort_values(['lat', 'lon'], ascending=[True, True])
        
        # MERGE two columns (n*1) into a new column (2n*1)
        # > ignore_index = True : 
        # >   have one continuous index numbers,
        # >     ignorning each of the two dfs original indices
        frames_Gexy=[df_exy_x,df_exy_y]
        df_Gexy=pd.concat(frames_Gexy,ignore_index=True) # merge the two dataFrames into one



        # SAVE a part of G-matrix (as in two different structures and then they will be merged later)

        # 1st structure = [Gexx(1) Geyy(1) Gexy(1) ... Gexx(HowManyCell) Geyy(HowManyCell) Gexy(HowManyCell)]   
        df_G["G_exx"+str(i)] = df_Gexx.loc[:,['velo']]
        df_G["G_eyy"+str(i)] = df_Geyy.loc[:,['velo']]
        df_G["G_exy"+str(i)] = df_Gexy.loc[:,['velo']]

In [33]:
print("DONE")

DONE


# **`STEP 2 :` Check the G matrix**

In [34]:
df_data = pd.read_csv("vel_obs_m_per_hr.gmt", header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')

In [35]:
df_data

,0,1,2,3,4,5,6
0,-44.75,60.25,0.003207,-0.014037,0,0,0
1,-44.25,60.25,0.005760,-0.012946,0,0,0
2,-43.75,60.25,0.007636,-0.011927,0,0,0
3,-45.75,60.75,-0.001760,-0.016655,0,0,0
4,-45.25,60.75,0.001120,-0.015782,0,0,0
...,...,...,...,...,...,...,...
2679,-27.25,83.25,-0.000458,0.006218,0,0,0
2680,-26.75,83.25,-0.000409,0.006327,0,0,0
2681,-26.25,83.25,-0.000362,0.006444,0,0,0
2682,-25.75,83.25,-0.000316,0.006553,0,0,0


In [36]:
if len(df_data)*2!=df_G.shape[0]:
    print("WARNING: Something went wrong!")
if HowManyCell*3!=df_G.shape[1]:
    print("WARNING: Something went wrong!")

<div class="alert alert-success">
    <b> If you don't see the WARNING, save G-matrix for the inversion. </b>
</div>

In [37]:
# SAVE G-matrix for the inversion
df_G.to_csv(outputFILE_G_matrix, index=None, float_format='%g')

In [38]:
df_G

,G_exx422,G_eyy422,G_exy422,G_exx423,G_eyy423,G_exy423,G_exx424,G_eyy424,G_exy424,G_exx461,...,G_exy870,G_exx871,G_eyy871,G_exy871,G_exx872,G_eyy872,G_exy872,G_exx873,G_eyy873,G_exy873
0,0.172,0.021,-0.054,0.057,0.017,-0.087,-0.204,-0.041,-0.066,0.025,...,0.000,0.000,-0.001,0.000,0.000,-0.001,0.000,0.000,-0.001,-0.000
1,0.143,0.004,-0.044,0.157,0.044,-0.084,-0.207,-0.052,-0.077,0.028,...,0.000,0.000,-0.001,0.000,0.000,-0.001,0.000,0.000,-0.001,-0.000
2,0.130,-0.004,-0.037,0.200,0.050,-0.077,-0.164,-0.045,-0.084,0.030,...,0.000,0.000,-0.001,0.000,0.000,-0.001,0.000,0.000,-0.001,-0.000
3,0.280,0.050,-0.025,-0.226,-0.045,-0.029,-0.186,0.006,-0.010,0.029,...,0.000,0.000,-0.001,0.000,0.000,-0.001,-0.000,0.000,-0.001,-0.000
4,0.270,0.036,-0.020,-0.087,-0.018,-0.031,-0.232,-0.014,-0.015,0.034,...,0.000,0.000,-0.001,0.000,0.000,-0.001,-0.000,0.000,-0.001,-0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5363,-0.000,0.002,0.000,-0.000,0.002,0.000,-0.000,0.002,0.000,-0.000,...,0.084,-0.008,0.030,0.110,-0.010,0.032,-0.054,-0.009,0.027,-0.148
5364,-0.000,0.002,0.000,-0.000,0.002,0.000,-0.000,0.002,0.000,-0.000,...,0.080,-0.008,0.028,0.104,-0.010,0.032,0.010,-0.009,0.029,-0.154
5365,-0.000,0.002,0.000,-0.000,0.002,0.000,-0.000,0.002,0.000,-0.000,...,0.077,-0.008,0.026,0.092,-0.009,0.032,0.066,-0.009,0.031,-0.146
5366,-0.000,0.002,0.000,-0.000,0.002,0.000,-0.000,0.002,0.000,-0.000,...,0.072,-0.007,0.024,0.085,-0.008,0.032,0.098,-0.009,0.032,-0.114
